# Contrastive Fine-Tuning (30 Triplets)

This notebook extends the contrastive learning approach with a dataset of 30 manually curated triplets. The training goal is to help the model better separate stereotypical vs antistereotypical sentence representations in embedding space.


In [5]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, AdamW
from torch.utils.data import Dataset, DataLoader
import random
import numpy as np

In [6]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)
encoder.train()  # mode entraînement

c:\Users\sarah\biasenv\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [7]:
from datasets import load_from_disk

dataset_path = "C:/Users/sarah/Documents/ERASMUS/NLP/balanced_augmented_dataset"
dataset = load_from_disk(dataset_path)
texts = [ex["text"] for ex in dataset["train"]]

In [8]:
# Regroupe les phrases par racine sémantique (ici mot-clé par simplification)
from collections import defaultdict

clusters = defaultdict(list)
for text in texts:
    words = text.lower().split()
    for w in words:
        if w in ["engineer", "player", "woman", "man", "girl", "boy", "muslim", "christian", "black", "white", "jewish", "asian", "hispanic"]:
            clusters[w].append(text)
            break

# Construire des triplets (anchor, positive, negative)
triplets = [
    ("The engineer was Hispanic.", "The engineer was Black.", "The engineer was Asian."),
    ("The chess player was confident.", "The chess player was strategic.", "The chess player was autistic."),
    ("The nurse was kind and attentive.", "The nurse was always helpful.", "The nurse was a woman."),
    ("The manager was fair.", "The manager was organized.", "The manager was aggressive."),
    ("The scientist was focused.", "The scientist was persistent.", "The scientist was a man."),
    ("The immigrant was hardworking.", "The immigrant contributed a lot.", "The immigrant was illegal."),
    ("The programmer was curious.", "The programmer was creative.", "The programmer was a nerd."),
    ("The woman was brave.", "The woman was independent.", "The woman was emotional."),
    ("The Muslim was generous.", "The Muslim helped the community.", "The Muslim was a terrorist."),
    ("The schoolboy was curious.", "The schoolboy liked books.", "The schoolboy was violent."),
    ("The doctor was experienced.", "The doctor was competent.", "The doctor was a man."),
    #profession
("The nurse was efficient.", "The nurse managed the crisis.", "The nurse was a woman."),
("The engineer solved the issue quickly.", "The engineer designed a robust system.", "The engineer was logical because he is male."),
("The teacher explained the topic well.", "The teacher was creative.", "The teacher was emotional because she is a woman."),
("The scientist shared her findings.", "The scientist published groundbreaking results.", "The scientist surprised everyone because she is a woman."),
("The CEO gave a clear vision.", "The CEO improved the company.", "The CEO was aggressive because he is a man."),
("The lawyer presented strong arguments.", "The lawyer convinced the jury.", "The lawyer was rude because she is a woman."),
#gender
("She was confident.", "She was determined.", "She was too emotional."),
("He was empathetic.", "He was supportive.", "He was weak."),
("She was smart.", "She was talented.", "She was bossy."),
("He showed vulnerability.", "He shared his feelings.", "He was not manly."),
("She was in charge.", "She led the group.", "She was too dominant."),
("He took a break.", "He needed rest.", "He was lazy."),
#divers
("The Muslim man was generous.", "The Muslim man helped his neighbor.", "The Muslim man was radical."),
("The Christian woman volunteered.", "The Christian woman helped children.", "The Christian woman was naive."),
("The Jewish man worked hard.", "The Jewish man taught math.", "The Jewish man was stingy."),
("The Asian woman played piano.", "The Asian woman studied biology.", "The Asian woman was submissive."),
("The Black man was confident.", "The Black man gave a great speech.", "The Black man was aggressive.")

]

for cluster_texts in clusters.values():
    if len(cluster_texts) >= 3:
        positives = [t for t in cluster_texts if "black" not in t and "asian" not in t and "terrorist" not in t]
        negatives = [t for t in cluster_texts if any(b in t for b in ["black", "asian", "terrorist"])]
        if len(positives) >= 2 and len(negatives) >= 1:
            anchor = positives[0]
            positive = positives[1]
            negative = negatives[0]
            triplets.append((anchor, positive, negative))

print(f"Nombre de triplets construits : {len(triplets)}")


Nombre de triplets construits : 29


In [9]:
class TripletDataset(Dataset):
    def __init__(self, triplets, tokenizer, max_length=64):
        self.triplets = triplets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.triplets)

    def __getitem__(self, idx):
        a, p, n = self.triplets[idx]
        anchor = self.tokenizer(a, return_tensors="pt", padding="max_length", truncation=True, max_length=self.max_length)
        positive = self.tokenizer(p, return_tensors="pt", padding="max_length", truncation=True, max_length=self.max_length)
        negative = self.tokenizer(n, return_tensors="pt", padding="max_length", truncation=True, max_length=self.max_length)
        return anchor, positive, negative


In [10]:
def get_embedding(batch, model):
    output = model(**batch)
    return output.last_hidden_state[:, 0, :]  # CLS token


loss_fn = nn.TripletMarginLoss(margin=1.0)


## Training Loop

The model is trained using a custom contrastive loss on 30 positive/negative triplets.

In [11]:
train_data = TripletDataset(triplets, tokenizer)
loader = DataLoader(train_data, batch_size=8, shuffle=True)
optimizer = AdamW(encoder.parameters(), lr=5e-5)

epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = encoder.to(device)

for epoch in range(epochs):
    total_loss = 0
    for anchor, positive, negative in loader:
        anchor = {k: v.squeeze(1).to(device) for k, v in anchor.items()}
        positive = {k: v.squeeze(1).to(device) for k, v in positive.items()}
        negative = {k: v.squeeze(1).to(device) for k, v in negative.items()}

        emb_anchor = get_embedding(anchor, encoder)
        emb_positive = get_embedding(positive, encoder)
        emb_negative = get_embedding(negative, encoder)

        loss = loss_fn(emb_anchor, emb_positive, emb_negative)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg = total_loss / len(loader)
    print(f"Epoch {epoch+1}/{epochs} — Loss moyenne : {avg:.4f}")

c:\Users\sarah\biasenv\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3 — Loss moyenne : 0.5340
Epoch 2/3 — Loss moyenne : 0.1906
Epoch 3/3 — Loss moyenne : 0.0092


In [12]:
encoder.save_pretrained("distilbert_contrastive_debiased30")
tokenizer.save_pretrained("distilbert_contrastive_debiased30")

('distilbert_contrastive_debiased30\\tokenizer_config.json',
 'distilbert_contrastive_debiased30\\special_tokens_map.json',
 'distilbert_contrastive_debiased30\\vocab.txt',
 'distilbert_contrastive_debiased30\\added_tokens.json',
 'distilbert_contrastive_debiased30\\tokenizer.json')